# Libraries

In [3]:
import requests
import pandas as pd

# Find Current Companies and CIK Codes

In [8]:
company_tickers_url = "https://www.sec.gov/files/company_tickers.json"
headers = {"User-Agent": "Debora Ramella debora.ramell@gmail.com"}

response = requests.get(company_tickers_url, headers=headers)


str

In [82]:
df = pd.DataFrame(data = [d for d in response.json().values()])
df.drop_duplicates(inplace=True)
df['cik_str'] = df['cik_str'].map(lambda x: (10-len(str(x)))*str(0) + str(x) if len(str(x))<10 else str(x))
df.set_index('cik_str',drop=True,inplace=True)


## Find basic information on companies

In [89]:
cik = '0000789019'
company_facts_url = f'https://data.sec.gov/submissions/CIK{cik}.json'
response = requests.get(company_facts_url, headers=headers)


In [122]:
sliced_keys=['cik','sic','sicDescription','tickers','exchanges','fiscalYearEnd']
sliced_dict = {key: response.json()[key] for key in sliced_keys}
df1 = pd.DataFrame(sliced_dict)
df1['cik'] = df1['cik'].map(lambda x: (10-len(str(x)))*str(0) + str(x) if len(str(x))<10 else str(x))
df1.set_index('cik',drop=True,inplace=True)
df1

# Download filings

In [127]:
from secedgar import filings, FilingType

# 10Q filings for Apple (ticker "aapl")
my_filings = filings(cik_lookup="aapl",
                     filing_type=FilingType.FILING_10Q,
                     user_agent="Debora Ramella debora.ramell@gmail.com")

scrape company txt files https://www.sec.gov/Archives/edgar/full-index/
load cik, filing types and urls
